### Đây là file chứa các model của chúng ta, các model nên được đặt ở một file riêng biệt rồi sau đó import vào những nơi cần dùng để cho việc quản lý các model phức tạp trở nên đơn giản và dễ dàng hơn

Phần import các module của pytorch

In [10]:
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

### 1. Ví dụ về tạo model

1.1 Model tùy chỉnh có hai lớp: Lớp tích chập (convolution layer) và lớp kết nối đầy đủ (fully-connected layer)

In [11]:
class CustomCNN(nn.Module):
    def __init__(self, input_channels=3, input_dim=224, num_class=10):
        super().__init__()
        self.conv_relu_bn_stack = nn.Sequential(
            nn.Conv2d(input_channels, 64, 7, 1, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            nn.Conv2d(64, 32, 5, 1, 2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.Conv2d(32, 16, 3, 1, 1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        
        self.fc_stack = {
            nn.Flatten(),
            nn.Linear(input_dim*input_dim*16, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Linear(1024, num_class)
        }
        
    def forward(self, x):
        logits = self.conv_relu_bn_stack(x)
        logits = self.fc_stack(logits)
        
        return logits

1.2 Model có sử dụng ResNet-50 để làm xương sống (backbone)

In [12]:
class BackboneResNet(nn.Module):
    def __init__(self, freeze_backbone=True, num_class=10):
        super().__init__()
        
        backbone = resnet50(weights=ResNet50_Weights.DEFAULT)
        
        # Gỡ lớp kết nối đầy đủ cuối cùng
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        
        # Đóng băng các tham số của backbone ResNet-50 để giữ lại tính năng lọc chi tiết của ResNet
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False
        
        # Thay thế lớp fc đã gỡ với một lớp fc mới
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_class)
        )
        
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(x)
        
        return x

### 2. Ví dụ về cách sử dụng model đã được tạo

2.1 Khởi tạo model custom để train trên dataset MNIST

In [13]:
input_channels = 1 # Các ảnh trong MNIST đều là ảnh dạng trắng đen
input_dim = 28     # Các ảnh trong MNIST có độ phân giải là 28x28
num_class = 10     # Các ảnh trong MNIST được chia thành 10 class là 10 sô viết bằng tay từ 0-9

MNIST_model = CustomCNN(input_channels, input_dim, num_class)

2.2 Khởi tạo model dùng ResNet làm backbone để train trên dataset MNIST

In [ ]:
freeze_backbone = True
num_class = 10     # Các ảnh trong MNIST được chia thành 10 class là 10 sô viết bằng tay từ 0-9

MNIST_model = BackboneResNet(freeze_backbone, num_class)